<a href="https://colab.research.google.com/github/waya0125/test/blob/main/%E3%81%A0%E3%82%81%E3%81%A0%E3%81%AD%E3%83%A1%E3%83%BC%E3%82%AB%E3%83%BC%EF%BC%8FDamedane_Maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# だめだねメーカー／Damedane Maker

順次に２つの▶ボタンを押して数十秒間経つとアップロードボタンが現れます。生成するにはまた数十秒ほどかかります。

Press the 2 ▶ buttons in order and an upload button will appear after tens of seconds. It takes another dozens of seconds to generate.

In [ ]:
!pip install PyYAML wget ipywidgets gdown ffmpy &> /dev/null
!git init -q .
!git remote add origin https://github.com/AliaksandrSiarohin/first-order-model
!git pull -q origin master

In [ ]:
import IPython.display
import PIL.Image
import cv2
import gdown
import imageio
import io
import ipywidgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path
import time
import warnings
from base64 import b64encode
from demo import load_checkpoints, make_animation
from ffmpy import FFmpeg
from google.colab import files
from skimage import img_as_ubyte
from skimage.transform import resize
warnings.filterwarnings("ignore")
if not os.path.isfile('original.mp4'):
    gdown.download('https://drive.google.com/uc?export=download&id=1M1ZXaMjUg2WAdf3OrQ3aBYC_3Dv15fqE', 'original.mp4', quiet=True)
if not os.path.isfile('vox-cpk.pth.tar'):
    gdown.download('https://drive.google.com/uc?export=download&id=1DbjXD2nS3jlyCWoJu2HGcLZZjhLC9a2J', 'vox-cpk.pth.tar', quiet=True)
reader = imageio.get_reader('original.mp4')
fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except:
    pass
reader.close()
driving_video = [resize(frame, (256, 256))[:, :, 0:3] for frame in driving_video]
input_image_widget = ipywidgets.Output(layout={'border': '1px solid gray', 'width': '256px', 'height': '256px'})
upload_input_image_button = ipywidgets.FileUpload(accept='image/*', multiple=False, layout={'width': '256px'})
output_video_widget = ipywidgets.Output(layout={'border': '1px solid gray', 'width': '256px', 'height': '256px'})
download_output_video_button = ipywidgets.Button(description='Download', layout={'width': '256px'})
download_output_video_button.disabled = True
display(ipywidgets.HBox([ipywidgets.VBox([input_image_widget, upload_input_image_button]), ipywidgets.VBox([output_video_widget, download_output_video_button])]))
def upload_image(change):
    for name, file in upload_input_image_button.value.items():
        input = file['content']
    if input is None:
        return
    download_output_video_button.disabled = True
    input = PIL.Image.open(io.BytesIO(input))
    w, h = input.size
    d = min(w, h)
    input.resize((256, 256), resample=PIL.Image.LANCZOS, box=((w - d) // 2, (h - d) // 2, (w + d) // 2, (h + d) // 2)).save('temp.png')
    input = resize(imageio.imread('temp.png'), (256, 256))[:, :, 0:3]
    input_image_widget.clear_output(wait=True)
    with input_image_widget:
        display(PIL.Image.open('temp.png'))
    output_video_widget.clear_output()
    generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='vox-cpk.pth.tar')
    predictions = make_animation(input, driving_video, generator, kp_detector, relative=True)
    imageio.mimsave('output.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
    FFmpeg(inputs={'output.mp4': None, 'original.mp4': None}, outputs={'damedane.mp4': '-c copy -y'}).run()
    with output_video_widget:
        display(IPython.display.HTML('<video controls src="data:video/mp4;base64,%s" />' % b64encode(open('damedane.mp4', 'rb').read()).decode()))
    download_output_video_button.disabled = False
def download_video(button):
    files.download('damedane.mp4')
upload_input_image_button.observe(upload_image, names='value')
download_output_video_button.on_click(download_video)

無編集の状態で動画を共有する場合は下の▶ボタンを押して約１分間お待ちください。サイズ 1920×1080 の動画が生成されます。

Press the ▶ button below and wait for about a minute for video sharing without editing. A video with size 1920×1080 will be generated.

In [ ]:
FFmpeg(inputs={'damedane.mp4': None}, outputs={'damedane_1920x1080.mp4': '-vf "scale=1080x1080:flags=lanczos,pad=1920:1080:420:0" -y'}).run()
files.download('damedane_1920x1080.mp4')